# **SQL Agent con LangChain**

Este notebook implementa un agente SQL que puede responder preguntas en lenguaje natural sobre los datos climáticos almacenados en la base de datos ClimateDB.

## Características:
- 🤖 Agente SQL con GPT-4o-mini de OpenAI
- 📊 Base de datos SQL Server con más de 1M de registros climáticos
- ⚡ Columnas temporales optimizadas (`year`, `month`, `day`, `hour`)
- 🔍 Índices para consultas eficientes por período y municipio
- 🌐 13 municipios de La Guajira, Colombia
- 📅 Datos desde 2015 hasta 2025

---


In [47]:
# ======================================================
# Project : GuajiraClimateAgents
# Author  : Eder Arley León Gómez
# GitHub  : https://github.com/ealeongomez
# License : MIT
# ======================================================

import sys, os
from pathlib import Path
from dotenv import load_dotenv
import pymssql
import pandas as pd

from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

# Cargar variables de entorno
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))
load_dotenv(PROJECT_ROOT / ".env")

print("✅ Imports completados")

✅ Imports completados


In [48]:
# Configuración de conexión
DB_CONFIG = {
    'server': os.getenv('DB_SERVER', 'localhost'),
    'port': os.getenv('DB_PORT', '1433'),
    'user': os.getenv('DB_USER', 'sa'),
    'password': os.getenv('DB_PASSWORD', 'SQLServer.1234'),
    'database': os.getenv('DB_NAME', 'ClimateDB')
}

print("ℹ️  Nota: La tabla climate_observations incluye columnas temporales:")
print("   • year, month, day, hour - Extraídas automáticamente de datetime")
print("   • Estas columnas optimizan consultas con filtros temporales")


ℹ️  Nota: La tabla climate_observations incluye columnas temporales:
   • year, month, day, hour - Extraídas automáticamente de datetime
   • Estas columnas optimizan consultas con filtros temporales


In [49]:
# Verificar conexión básica
try:
    conn = pymssql.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("SELECT DB_NAME(), COUNT(*) FROM climate_observations")
    db_name, count = cursor.fetchone()
    print(f"✅ Conectado a: {db_name}")
    print(f"   Total de registros: {count:,}")
    conn.close()
except Exception as e:
    print(f"❌ Error de conexión: {e}")

✅ Conectado a: ClimateDB
   Total de registros: 1,139,112


In [50]:
# Crear URI de conexión para SQLAlchemy
# Nota: pymssql es más simple que pyodbc (no necesita driver ODBC)
MSSQL_URI = (
    f"mssql+pymssql://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['server']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# Crear objeto SQLDatabase de LangChain
db = SQLDatabase.from_uri(MSSQL_URI)

print("✅ SQLDatabase creado")
print(f"   Dialect: {db.dialect}")
print(f"   Tables: {db.get_usable_table_names()}")

✅ SQLDatabase creado
   Dialect: mssql
   Tables: ['climate_observations']


In [51]:
# Ver el schema de la tabla
print(db.get_table_info(['climate_observations']))


CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	municipio	datetime	wind_speed_10m	wind_direction_10m	temperature_2m	relative_humidity_2m	precipitation	created_at	year	month	day	hour
1	albania	2015-12-21 00:00:00	13.0	95	24.3	89	0.0	2025-12-23 16:40:00.290000	2015	12	21	0
2	albania	2015-12-21 01:00:00	13.7	95	24.2	89	0.0	202

In [52]:
# Ejecutar una consulta SQL directa
query = """
    SELECT 
        municipio,
        COUNT(*) as total_registros,
        AVG(wind_speed_10m) as velocidad_promedio_viento,
        AVG(temperature_2m) as temperatura_promedio
    FROM climate_observations
    GROUP BY municipio
    ORDER BY velocidad_promedio_viento DESC
"""

result = db.run(query)
print("📊 Resultado de consulta directa:\n")
print(result)

📊 Resultado de consulta directa:

[('manaure', 87624, 17.6246941477221, 28.551881904501), ('uribia', 87624, 16.781839450378897, 28.74946589975343), ('riohacha', 87624, 15.53369510636356, 28.320914361362135), ('maicao', 87624, 15.34921254450833, 27.639090888341148), ('la_jagua_del_pilar', 87624, 14.688101433397234, 28.37402309869448), ('san_juan_del_cesar', 87624, 13.340719437596992, 28.06255706199215), ('distraccion', 87624, 13.237238656075961, 27.878197754039938), ('fonseca', 87624, 12.658547886423804, 28.009258878846097), ('el_molino', 87624, 12.437099424815129, 27.690288048936406), ('barrancas', 87624, 11.830404911896297, 27.8707956724185), ('albania', 87624, 9.339294029033159, 27.28953254816031), ('hatonuevo', 87624, 9.33636332511637, 26.887922258741863), ('mingueo', 87624, 8.296277275632269, 26.424275312699713)]


In [53]:
# Inicializar el modelo
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Usa gpt-4o-mini para ser más económico
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Modelo LLM inicializado")
print(f"   Model: {llm.model_name}")

✅ Modelo LLM inicializado
   Model: gpt-4o-mini


In [54]:
# Crear toolkit con las herramientas SQL
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Ver las herramientas disponibles
tools = toolkit.get_tools()

print(f"🔧 Herramientas disponibles ({len(tools)}):\n")
for tool in tools:
    print(f"   • {tool.name}: {tool.description[:80]}...")

🔧 Herramientas disponibles (4):

   • sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from ...
   • sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and...
   • sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the data...
   • sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Alwa...


In [55]:
# Crear el agente SQL
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit, 
    verbose=True,
    agent_type="openai-tools", 
)

print("✅ Agente SQL creado y listo para usar")

✅ Agente SQL creado y listo para usar


In [56]:
# Ejemplo 1: Consulta simple
pregunta1 = "¿Cuál es la velocidad promedio del viento en Riohacha?"

print(f"🤔 Pregunta: {pregunta1}\n")
print("="*80)

respuesta1 = agent_executor.invoke({"input": pregunta1})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta1['output']}")

🤔 Pregunta: ¿Cuál es la velocidad promedio del viento en Riohacha?



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	municipio	datetime	wind_

In [57]:
# Ejemplo 2: Comparación
pregunta2 = "Compara la temperatura promedio entre Maicao, Uribia y Riohacha"

print(f"🤔 Pregunta: {pregunta2}\n")
print("="*80)

respuesta2 = agent_executor.invoke({"input": pregunta2})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta2['output']}")

🤔 Pregunta: Compara la temperatura promedio entre Maicao, Uribia y Riohacha



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	municipio	datet

In [58]:
# Ejemplo 3: Ranking
pregunta3 = "¿Cuáles son los 5 municipios con mayor velocidad promedio del viento?"

print(f"🤔 Pregunta: {pregunta3}\n")
print("="*80)

respuesta3 = agent_executor.invoke({"input": pregunta3})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta3['output']}")

🤔 Pregunta: ¿Cuáles son los 5 municipios con mayor velocidad promedio del viento?



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	municipio

In [59]:
# Ejemplo 4: Análisis temporal
pregunta4 = "¿Cuál es el promedio de velocidad del viento en Uribia durante enero de 2025?"

print(f"🤔 Pregunta: {pregunta4}\n")
print("="*80)

respuesta4 = agent_executor.invoke({"input": pregunta4})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta4['output']}")

🤔 Pregunta: ¿Cuál es el promedio de velocidad del viento en Uribia durante enero de 2025?



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	m

In [60]:
# Ejemplo 5: Análisis más complejo
pregunta5 = """
En qué mes del 2024 hubo mayor velocidad de viento promedio 
en todos los municipios combinados?
"""

print(f"🤔 Pregunta: {pregunta5}\n")
print("="*80)

respuesta5 = agent_executor.invoke({"input": pregunta5})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta5['output']}")

🤔 Pregunta: 
En qué mes del 2024 hubo mayor velocidad de viento promedio 
en todos los municipios combinados?




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_obs

## **Ejemplos Adicionales - Aprovechando columnas temporales**

Las columnas `year`, `month`, `day` y `hour` permiten consultas más eficientes sin necesidad de usar funciones SQL como `YEAR()`, `MONTH()`, etc.


In [61]:
# Ejemplo 6: Consulta usando columnas temporales directamente
pregunta6 = """
Dame el promedio de temperatura para cada hora del día (0-23) 
en el municipio de Riohacha durante el año 2024.
Usa la columna hour que ya existe en la tabla.
"""

print(f"🤔 Pregunta: {pregunta6}\n")
print("="*80)

respuesta6 = agent_executor.invoke({"input": pregunta6})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta6['output']}")


🤔 Pregunta: 
Dame el promedio de temperatura para cada hora del día (0-23) 
en el municipio de Riohacha durante el año 2024.
Usa la columna hour que ya existe en la tabla.




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE

In [62]:
# Ejemplo 7: Análisis por mes usando columna month
pregunta7 = """
¿En qué mes (1-12) del 2024 se registró la mayor precipitación total
en todos los municipios combinados? Usa la columna month.
"""

print(f"🤔 Pregunta: {pregunta7}\n")
print("="*80)

respuesta7 = agent_executor.invoke({"input": pregunta7})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta7['output']}")


🤔 Pregunta: 
¿En qué mes (1-12) del 2024 se registró la mayor precipitación total
en todos los municipios combinados? Usa la columna month.




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)

In [63]:
# Ejemplo 8: Comparación entre años
pregunta8 = """
Compara el promedio de velocidad del viento en Manaure 
entre el año 2023 y 2024. Usa la columna year.
"""

print(f"🤔 Pregunta: {pregunta8}\n")
print("="*80)

respuesta8 = agent_executor.invoke({"input": pregunta8})

print("\n" + "="*80)
print(f"💡 Respuesta: {respuesta8['output']}")


🤔 Pregunta: 
Compara el promedio de velocidad del viento en Manaure 
entre el año 2023 y 2024. Usa la columna year.




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from clima

In [64]:
conn.close()
print("\n👋 Conexión cerrada")
print("\n" + "="*80)
print("✅ Notebook completado exitosamente")
print("="*80)



👋 Conexión cerrada

✅ Notebook completado exitosamente


In [65]:
def consultar_clima(pregunta: str, verbose: bool = False) -> str:
    """Helper function para hacer consultas rápidas.
    
    Args:
        pregunta: Pregunta en lenguaje natural.
        verbose: Si mostrar el proceso de razonamiento.
    
    Returns:
        Respuesta del agente.
    """
    # Crear agente con o sin verbose
    agent = create_sql_agent(
        llm=llm,
        toolkit=toolkit, 
        verbose=True,
        agent_type="openai-tools"
        )
    
    resultado = agent.invoke({"input": pregunta})
    return resultado['output']

# Probar la función
print("🧪 Probando función helper:\n")
respuesta = consultar_clima(
    "¿Cuál es la temperatura máxima registrada en toda la base de datos?",
    verbose=False
)
print(f"💡 {respuesta}")

🧪 Probando función helper:



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	municipio	datetime	wind_speed_10m	wind_direction_10m	temperature

In [66]:
# Ejecutar múltiples consultas
preguntas = [
    "¿Cuántos municipios hay en la base de datos?",
    "¿Cuál es el rango de fechas disponible?",
    "¿Hay datos faltantes (NULL) en wind_speed_10m?",
]

print("📋 Ejecutando múltiples consultas:\n")
print("="*80)

for i, pregunta in enumerate(preguntas, 1):
    print(f"\n{i}. {pregunta}")
    respuesta = consultar_clima(pregunta, verbose=False)
    print(f"   💡 {respuesta}")
    print("-"*80)

📋 Ejecutando múltiples consultas:


1. ¿Cuántos municipios hay en la base de datos?


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


climate_observations
Invoking: `sql_db_schema` with `{'table_names': 'climate_observations'}`



CREATE TABLE climate_observations (
	id BIGINT NOT NULL IDENTITY(1,1), 
	municipio NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	datetime DATETIME2 NOT NULL, 
	wind_speed_10m FLOAT(53) NULL, 
	wind_direction_10m INTEGER NULL, 
	temperature_2m FLOAT(53) NULL, 
	relative_humidity_2m INTEGER NULL, 
	precipitation FLOAT(53) NULL, 
	created_at DATETIME2 NULL DEFAULT (getutcdate()), 
	year AS ((datepart(year,[datetime]))) PERSISTED, 
	month AS ((datepart(month,[datetime]))) PERSISTED, 
	day AS ((datepart(day,[datetime]))) PERSISTED, 
	hour AS ((datepart(hour,[datetime]))) PERSISTED, 
	CONSTRAINT [PK__climate___3213E83FE7BE4ED9] PRIMARY KEY CLUSTERED (id)
)

/*
3 rows from climate_observations table:
id	municipi